In [1]:
import pandas as pd
import numpy as np

In [2]:
#Data
df = pd.DataFrame({'Sequence' : ["<AB(FG)CD>", "<BGD>", "<BFG(AB)>", "<F(AB)CD>", "<A(BC)GF(DE)>"]}, [1, 2, 3, 4, 5])
df.head()

,Sequence
1,<AB(FG)CD>
2,<BGD>
3,<BFG(AB)>
4,<F(AB)CD>
5,<A(BC)GF(DE)>


In [3]:
min_supportCount = 2

In [4]:
#Getting All Unique events in All Sequences
def uniqueSeq (df):
    dic = {}
    arr = []
    for i in df.index:
        s = set(df['Sequence'][i])
        for j in s:
            dic[j] = 0
            
    for ele in dic.keys():
        arr.append(ele)
    
    if (')' in arr):
        arr.remove(')')
    if ('(' in arr):
        arr.remove('(')
        
    arr.remove('<')
    arr.remove('>')
    return arr

uniqueSeq(df)

['G', 'B', 'C', 'A', 'F', 'D', 'E']

In [5]:
#Splitting the Subsequence into Array
def splittingElement(element) :
    arr = []
    sequence = element
    isBrace = False
    s = ""
    for ele in sequence:
        if(ele == '('):
            isBrace = True
            continue
        elif (ele == ')'):
            isBrace = False
            arr.append(s)
            s = ""
            continue
        else:
            if (isBrace == True):
                s += ele
            else:
                arr.append(ele)
    return arr

splittingElement('(AF)GB')

['AF', 'G', 'B']

In [6]:
#Helper Function to loop over the Sequence
def seqLooping(SeqArr, ele, index):  
    for k in range(index + 1, len(SeqArr)):  
        for c in range(0, len(SeqArr[k])): 
            if (ele == SeqArr[k][c]): 
                return k
                    
    
    return -2

In [7]:
#Check if the Sequence is a Subsequence of the Original Sequence
def isSubSequence(SeqArr, elementArr): 
    thirdIndex = -1
    x = -1
    for i in range(0, len(elementArr)): 
        firstIndex = -1
        secIndex = -1
        j = 0
        
        while(j < len(elementArr[i])): 
            if(firstIndex == -1):
                firstIndex = seqLooping(SeqArr, elementArr[i][j], thirdIndex) 
            else:
                secIndex = seqLooping(SeqArr, elementArr[i][j], firstIndex - 1)  
                
                if(firstIndex == secIndex):  
                    firstIndex = secIndex  
                else:
                    if(firstIndex<len(SeqArr)-1 and firstIndex!=-2):
                        thirdIndex = firstIndex #0
                        firstIndex = secIndex = -1
                        j = -1
                    else:    
                        return 0
                    
            if(firstIndex == -2):
                return 0
            j+=1    
        
        
        
        if(firstIndex == -2):
            return 0
        elif(secIndex != -1 and firstIndex != secIndex):
            return 0
        
        if(thirdIndex != -1):
            if(thirdIndex > firstIndex):
                return 0
            else: 
                thirdIndex = firstIndex 
        else:
            thirdIndex = firstIndex
        
        
                
        
        x = i
        
           
    if(x == len(elementArr) - 1):        
        return 1
    else:
        return 0
            
isSubSequence(['AC', 'AB'], ['AB'])

1

In [8]:
#Turning ALL The Origingal Sequences in Data into Array
def fromSeqToArr(df):
    arr = []
    
    for i in df.index:
        x = splittingElement(df['Sequence'][i])
        if '<' in x:
            x.remove('<')
        if '>' in x:
            x.remove('>')
        arr.append(x)
    
    return arr  


fromSeqToArr(df)

[['A', 'B', 'FG', 'C', 'D'],
 ['B', 'G', 'D'],
 ['B', 'F', 'G', 'AB'],
 ['F', 'AB', 'C', 'D'],
 ['A', 'BC', 'G', 'F', 'DE']]

In [9]:
#Calculating Support for a Specific Element
def calcSupport(seqsArr, element):
    elementArr = splittingElement(element)
    count = 0
    
    for i in range(0, len(seqsArr)):
        x = isSubSequence(seqsArr[i], elementArr)
        count += x
 
    return count

seqArr = fromSeqToArr(df)
calcSupport(seqArr, '(AB)')

2

In [10]:
#Gettting All Frequent 1-Itemsets
def freq_1Item (seqsArr):
    uniqueItems = sorted(uniqueSeq(df))
    oneItemSeq = []
    sups = []
    
    oneItemSeqDF = pd.DataFrame()
    
    for item in uniqueItems:
        sup = calcSupport(seqsArr, item)
        if(sup >= min_supportCount):
            sups.append(sup)
            oneItemSeq.append(item)
            
    oneItemSeqDF['1-Frequent Itemset'] = oneItemSeq
    oneItemSeqDF['Support Count'] = sups
    
    return oneItemSeqDF

In [11]:
#Getting the Temporal Join for 2-itemsets canditates
def _2ItemJoinTemporal ():
    oneItemSeq = list(freq_1Item(seqArr)['1-Frequent Itemset'])
    arr = []
    
    for i in range(0, len(oneItemSeq)):
        for j in range(0, len(oneItemSeq)):
            arr.append(oneItemSeq[i]+oneItemSeq[j])
            
    return arr

In [12]:
#Getting the Non-Temporal Join for 2-itemsets canditates
def _2ItemJoinNON_Temporal():
    oneItemSeq = list(freq_1Item(seqArr)['1-Frequent Itemset'])
    arr = []
    
    for i in range(0, len(oneItemSeq)):
        for j in range(i, len(oneItemSeq)):
            if(i == j):
                continue
            arr.append('('+oneItemSeq[i]+oneItemSeq[j]+')')
    
    return arr

In [13]:
#Frequent 2-Itemsets
def freq_2Item (seqArr):
    twoItemSeq = _2ItemJoinTemporal() + _2ItemJoinNON_Temporal()
    twoFreqItemSeq = []
    sups = []
    twoFreqItemSeqDF = pd.DataFrame()
    
    for item in twoItemSeq:
        sup = calcSupport(seqArr, item)
        if(sup >= min_supportCount):
            sups.append(sup)
            twoFreqItemSeq.append(item)
    
    twoFreqItemSeqDF['2-Frequent Itemset'] = twoFreqItemSeq
    twoFreqItemSeqDF['Support Count'] = sups 
    
    return twoFreqItemSeqDF

In [14]:
#Removing the Single Bracket of an Element
def removeSingleBrackets(element):
    for i in range(0,len(element)-2):
        if(element[i]=='('):
            if(element[i+2]==')'):
                if(i+3<len(element)):
                    element=element[:i]+element[i+1]+element[i+3:]
                else:
                    element=element[:i]+element[i+1]
                    break  
    return element

In [15]:
#Joining 3 or more Itemsets
#3 or more item-set 
def _NItemJoinTemporal(twoItemSeq):
    arr = []
    
    for i in range(0, len(twoItemSeq)):
        for j in range(0, len(twoItemSeq)):
            if(i==j):
                continue
            if(not(twoItemSeq[i][0]=='(' and twoItemSeq[j][len(twoItemSeq[j])-1:]==')')):   
                if(twoItemSeq[i][1:] ==twoItemSeq[j][:len(twoItemSeq[j]) - 1]):
                    arr.append(twoItemSeq[i]+twoItemSeq[j][len(twoItemSeq[j]) - 1])
                    
        
                elif(twoItemSeq[i][0]=='(' and (twoItemSeq[j][len(twoItemSeq[j]) - 1]!=')')):
                    tmp=twoItemSeq[i]
                    for k in range(1, len(tmp)):
                        if(twoItemSeq[i][k]==')'):
                            break
                        tmp=twoItemSeq[i]
                        tmp=tmp[:k]+tmp[k+1:]
                        if (removeSingleBrackets(tmp)==twoItemSeq[j][:len(twoItemSeq[j]) - 1]):
                            arr.append(twoItemSeq[i]+twoItemSeq[j][len(twoItemSeq[j]) - 1])
                            
                elif(twoItemSeq[i][0]!='(' and (twoItemSeq[j][len(twoItemSeq[j]) - 1]==')')):
                    tmp=twoItemSeq[j]
                    for k in range(len(tmp)-1,0,-1):
                        if(tmp[k]=='('):
                            break
                        tmp=twoItemSeq[j]
                        tmp=tmp[:k]+tmp[k+1:]
                        if (twoItemSeq[i][1:len(twoItemSeq[i])]==removeSingleBrackets(tmp)):
                            arr.append(twoItemSeq[i][0]+twoItemSeq[j])
    arr = list(dict.fromkeys(arr))
    return arr
# < a b c>   <(b c d)>  <a b (cd) 

In [16]:
#Claculating the Frequent N-Itemsets
def freq_NItem (seqArr, NItemSeq):
    NFreqItemSeq = []
    sups = []
    NFreqItemSeqDF = pd.DataFrame()
    
    for item in NItemSeq:
        sup = calcSupport(seqArr, item)
        if(sup >= min_supportCount):
            sups.append(sup)
            NFreqItemSeq.append(item)
    
    NFreqItemSeqDF['N-Frequent Itemset'] = NFreqItemSeq
    NFreqItemSeqDF['Support Count'] = sups
    
    return NFreqItemSeqDF

In [17]:
#Calculating All Frequent Itemsets for All Levels in Data
def calcAllSeq():
    seqArr = fromSeqToArr(df)
    firstFreq = freq_1Item(seqArr)
    twoFreq = freq_2Item(seqArr)
    
    NfreqArray = []
    NfreqArray.append(firstFreq)
    NfreqArray.append(twoFreq)
    
    firstTime = True
    
    while True:
        if firstTime:
            firstTime = False
            twoFreq = list(twoFreq['2-Frequent Itemset'])
        else:
            twoFreq = list(twoFreq['N-Frequent Itemset'])
        arr = _NItemJoinTemporal(twoFreq)
        arr = freq_NItem(seqArr, arr)
        if(arr.shape[0] >= 1):
            twoFreq = arr
            NfreqArray.append(arr)
        else:
            break


    return NfreqArray

In [18]:
freqItemsets = calcAllSeq()
#first Level
freqItemsets[0]

,1-Frequent Itemset,Support Count
0,A,4
1,B,5
2,C,3
3,D,4
4,F,4
5,G,4


In [19]:
#Second Level
freqItemsets[1]

,2-Frequent Itemset,Support Count
0,AB,2
1,AC,3
2,AD,3
3,AF,2
4,AG,2
5,BC,2
6,BD,4
7,BF,3
8,BG,4
9,CD,3


In [20]:
#Third Level
freqItemsets[2]

,N-Frequent Itemset,Support Count
0,ABD,2
1,ABF,2
2,ABG,2
3,ACD,3
4,AFD,2
5,AGD,2
6,BCD,2
7,BFD,2
8,BGD,3
9,F(AB),2


In [21]:
#Fourth Level
freqItemsets[3]

,N-Frequent Itemset,Support Count
0,ABFD,2
1,ABGD,2


In [22]:
freqItemsets

[  1-Frequent Itemset  Support Count
 0                  A              4
 1                  B              5
 2                  C              3
 3                  D              4
 4                  F              4
 5                  G              4,
    2-Frequent Itemset  Support Count
 0                  AB              2
 1                  AC              3
 2                  AD              3
 3                  AF              2
 4                  AG              2
 5                  BC              2
 6                  BD              4
 7                  BF              3
 8                  BG              4
 9                  CD              3
 10                 FA              2
 11                 FB              2
 12                 FC              2
 13                 FD              3
 14                 GD              3
 15               (AB)              2,
    N-Frequent Itemset  Support Count
 0                 ABD              2
 1               

In [23]:
# def calculateConfidance(seqArr,arr):
#     for i in arr:
#         if(len(i)>=2):
#             support=calcSupport(seqArr,i)
#             arr2=splittingElement(i)
#             for j in arr2:
#                 support2=calcSupport(seqArr,j)
#                 if(support2!=0):
#                     if (support/support2>=0.8):
#                         print("i ",i)
# seqArr = fromSeqToArr(df)
# cc=calcAllSeq()
# calculateConfidance(seqArr,cc)   